In [ ]:
import numpy as np
from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt
import networkx as nx


In [ ]:
def generate_storage_locations(rows, cols, rack_width=1000, rack_height=4000, walkway_width=2000):
    """
    Generates rack polygons and storage location vertices.
    Each rack has 8 shelves and 10 storage locations per shelf.
    :param rows: Number of rows of racks
    :param cols: Number of columns of racks
    :param rack_width: Width of each rack in mm
    :param rack_height: Height of each rack in mm
    :param walkway_width: Width of walkways between racks in mm
    :return: (list of rack polygons, list of storage locations with labels)
    """
    rack_polygons = []
    storage_locations = []  # List of (coordinates, label) tuples

    for row in range(rows):
        for col in range(cols):
            # Bottom-left corner of the rack
            x_start = col * (rack_width + walkway_width)
            y_start = row * (rack_height + walkway_width)

            # Create a polygon for the rack
            rack_polygon = Polygon([
                (x_start, y_start),  # Bottom-left
                (x_start, y_start + rack_height),  # Top-left
                (x_start + rack_width, y_start + rack_height),  # Top-right
                (x_start + rack_width, y_start)  # Bottom-right
            ])
            rack_polygons.append(rack_polygon)

            # Generate storage locations for this rack
            rack_id = f"{row + 1:02}{col + 1:02}"  # Rack identifier
            shelf_height = rack_height / 8  # Divide rack height into 8 shelves
            location_width = rack_width / 10  # Divide rack width into 10 locations

            for shelf_idx, shelf_letter in enumerate("ABCDEFGH"):  # 8 shelves
                for loc_idx in range(10):  # 10 locations per shelf
                    x_loc = x_start + loc_idx * location_width + location_width / 2
                    y_loc = y_start + shelf_idx * shelf_height + shelf_height / 2
                    label = f"{rack_id}{shelf_letter}{loc_idx + 1:02}"
                    storage_locations.append(((x_loc, y_loc), label))

    return rack_polygons, storage_locations


def add_picker_depot(storage_locations, depot_x=0, depot_y=-2000):
    """
    Adds the picker depot as a storage location.
    :param storage_locations: List of (coordinates, label) tuples for storage locations
    :param depot_x: X-coordinate of the depot
    :param depot_y: Y-coordinate of the depot
    :return: Updated storage locations, depot label
    """
    depot_label = "Depot"
    storage_locations.append(((depot_x, depot_y), depot_label))
    return storage_locations, depot_label


In [ ]:
def generate_weighted_adjacency_matrix_with_labels(storage_locations):
    """
    Generates the weighted adjacency matrix for all storage locations.
    :param storage_locations: List of (coordinates, label) tuples
    :return: Weighted adjacency matrix, mapping of labels to indices
    """
    N = len(storage_locations)
    weightedAdjMat = np.zeros((N, N))  # Initialize matrix
    label_to_index = {label: idx for idx, (_, label) in enumerate(storage_locations)}

    for i in range(N):
        for j in range(i + 1, N):  # Upper triangular matrix
            coord_i, _ = storage_locations[i]
            coord_j, _ = storage_locations[j]

            # Compute Euclidean distance
            distance = np.sqrt((coord_i[0] - coord_j[0])**2 + (coord_i[1] - coord_j[1])**2)
            weightedAdjMat[i, j] = distance
            weightedAdjMat[j, i] = distance  # Symmetric

    return weightedAdjMat, label_to_index
